In [2]:
import torch
import numpy as np
x = np.array([[1,2],[3,4.]])
x
y = torch.tensor(x)
y
print(x.dtype, y.dtype)

t3 = torch.tensor([
    [[11,12,13,14],
     [13,14,15,16]],
    [[15,16,17,18],
     [16,17,18,19.]],
    [[15,16,17,18],
     [16,17,18,19.]]
])
t3
t3.shape

float64 torch.float64


torch.Size([3, 2, 4])

In [3]:
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)
y = w * x + b
y

tensor(17., grad_fn=<AddBackward0>)

In [4]:
y.backward()
print('dy/dx', x.grad)
print('dy/dw', w.grad)
print('dy/db', b.grad)

dy/dx None
dy/dw tensor(3.)
dy/db tensor(1.)


In [5]:
import numpy as np
import torch
import math
#region, temp,rain,humidity,apples,oranges
#kanto, 73, 67, 43, 56, 70
#johto, 91, 88, 64 ,81, 101
#hoen, 87, 134, 58, 119, 133
#sinnoh, 102, 43,37, 22, 37
#unova, 69, 96, 70, 103, 119
#yield_apple = w11 * temp + w12 * rainfall + w13 * humidity + b1
#yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

inputs = np.array([
[73,67,43],
[91,88,64],
[87,134,58],
[102,43,37],
[69,96,70]], dtype='float32')
targets = np.array([
[56,70],
[81,101],
[119,133],
[22,37],
[103,119]], dtype='float32')
#avg 76.2, 92
inputs = torch.tensor(inputs)
targets = torch.from_numpy(targets)
print(inputs.shape)
print(targets.shape)
print(inputs)
print(targets)



torch.Size([5, 3])
torch.Size([5, 2])
tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


torch.randn creates a tensor with the given shape, with elements pickedrandomly from a normal distribution with mean 0 and stand deviation 1.
Model is a function that performs a matrix multiplication of the inputs and the weights w (transposed) and adds the bias b (replicated for each observation)

In [6]:
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)
print(w)
print(b)
def model(x):
    return x @ w.t() + b

tensor([[-0.9184, -2.1439, -0.0340],
        [ 0.0606, -0.0868,  0.0777]], requires_grad=True)
tensor([-0.3817, -0.4314], requires_grad=True)


@ represents matrix mulitiplication, and .t metod returns transpose of a tensor.
matrix obtained by passing the input data into the model is a set of predictions for the target vars.

In [7]:
preds = model(inputs)
print(preds)

tensor([[-212.5319,    1.5186],
        [-274.8000,    2.4184],
        [-369.5440,   -2.2839],
        [-187.5075,    4.8939],
        [-271.9502,    0.8565]], grad_fn=<AddBackward0>)


In [8]:
diff = preds - targets
print(diff)
diff_sqr = diff * diff
print(diff_sqr)
manual_mse = torch.sum(diff_sqr) / diff.numel()
print(manual_mse)
print(math.sqrt(manual_mse))

tensor([[-268.5319,  -68.4814],
        [-355.8000,  -98.5816],
        [-488.5440, -135.2840],
        [-209.5075,  -32.1061],
        [-374.9502, -118.1435]], grad_fn=<SubBackward0>)
tensor([[ 72109.3906,   4689.7085],
        [126593.6328,   9718.3281],
        [238675.2188,  18301.7480],
        [ 43893.3750,   1030.7993],
        [140587.6719,  13957.8945]], grad_fn=<MulBackward0>)
tensor(66955.7812, grad_fn=<DivBackward0>)
258.75815204549593


In [9]:
def mse(t1,t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()
loss = mse(preds, targets)
print(loss)

tensor(66955.7812, grad_fn=<DivBackward0>)


.nume1 method returns the nummber of elements in a tensor
Loss is quadratic function of weights and bieases

After calling backward(), the gradients are stored in the .grad attribute of the tensors.

gradients are stored in the .grad property of the respective tensors. Note derivative of the loss w.r.t. the weights matrix is itself a matrix, with same dimentsion

In [10]:
preds = model(inputs)
loss = mse(preds, targets)
loss.backward()
print(w)
print(b)
print(w.grad)
print(b.grad)
print(loss)


tensor([[-0.9184, -2.1439, -0.0340],
        [ 0.0606, -0.0868,  0.0777]], requires_grad=True)
tensor([-0.3817, -0.4314], requires_grad=True)
tensor([[-28345.0547, -31954.1953, -19330.3828],
        [ -7433.2993,  -8822.7656,  -5311.6729]])
tensor([-339.4667,  -90.5193])
tensor(66955.7812, grad_fn=<DivBackward0>)


In [11]:
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5
  w.grad.zero_()
  b.grad.zero_()
print(w)
print(b)

tensor([[-6.3497e-01, -1.8244e+00,  1.5932e-01],
        [ 1.3495e-01,  1.4096e-03,  1.3082e-01]], requires_grad=True)
tensor([-0.3783, -0.4305], requires_grad=True)


In [12]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(45471.4609, grad_fn=<DivBackward0>)


Significant Reduction of Loss!!!

In [13]:
for i in range(1000):
  preds = model(inputs)
  loss = mse(preds, targets)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [14]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)
print(math.sqrt(loss))
print(b)
print(w)

tensor(4.4609, grad_fn=<DivBackward0>)
2.112086270239298
tensor([-0.3727, -0.4299], requires_grad=True)
tensor([[-0.4402,  0.7721,  0.8775],
        [-0.2532,  0.8666,  0.7272]], requires_grad=True)


========================================================

In [15]:
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
inputs = np.array([
[73,67,43],
[91,88,64],
[87,134,58],
[73,67,43],
[91,88,64],
[87,134,58],
[102,43,37],
[69,96,70],
[102,43,37],
[69,96,70],
[73,67,43],
[91,88,64],
[87,134,58],
[102,43,37],
[69,96,70]], dtype='float32')

targets = np.array([
[56,70],
[81,101],
[119,133],
[56,70],
[81,101],
[119,133],
[22,37],
[103,119],
[22,37],
[103,119],
[56,70],
[81,101],
[119,133],
[22,37],
[103,119]], dtype='float32')
#avg 76.2, 92
inputs = torch.tensor(inputs)
targets = torch.from_numpy(targets)
print(inputs.shape)
print(targets.shape)
print(inputs)
print(targets)

torch.Size([15, 3])
torch.Size([15, 2])
tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [16]:
#define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

TDset allows access to small section of training data using array [0:3]. returning a typle pair in which 1st element contains input variables for selected rows and 2nd, the target

In [17]:
batch_size = 5
#manual
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
for xb, yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [ 69.,  96.,  70.],
        [ 91.,  88.,  64.]])
tensor([[103., 119.],
        [ 22.,  37.],
        [ 56.,  70.],
        [103., 119.],
        [ 81., 101.]])


In [18]:
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

preds = model(inputs)
print(preds)

Parameter containing:
tensor([[-0.3273,  0.5289, -0.0542],
        [-0.1197,  0.4837, -0.0051]], requires_grad=True)
Parameter containing:
tensor([0.0767, 0.2993], requires_grad=True)
tensor([[  9.2905,  23.7442],
        [ 13.3684,  31.6388],
        [ 39.3337,  54.3965],
        [  9.2905,  23.7442],
        [ 13.3684,  31.6388],
        [ 39.3337,  54.3965],
        [-12.5714,   8.6943],
        [ 24.4760,  38.1119],
        [-12.5714,   8.6943],
        [ 24.4760,  38.1119],
        [  9.2905,  23.7442],
        [ 13.3684,  31.6388],
        [ 39.3337,  54.3965],
        [-12.5714,   8.6943],
        [ 24.4760,  38.1119]], grad_fn=<AddmmBackward0>)


In [19]:
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(4093.6907, grad_fn=<MseLossBackward0>)


In [20]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

1. Generate predictions
2. Calculate the loss
3. Compute gradients with respect to Weights and Biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset gradients to zero

In [21]:
def fit(num_epochs, model, loss_fn, opt):
  for epoch in range(num_epochs):
    for xb, yb in train_dl:
      pred = model(xb)
      loss = loss_fn(pred, yb)
      loss.backward()
      opt.step()
      opt.zero_grad()
    if (epoch+1) % 10 == 0:
      print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Stochastic Gradient Descent (stochastic due to batches instead of single group)

.parameters method returns list containing all weights and bias matrices

In [22]:
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 62.1032
Epoch [20/100], Loss: 92.1569
Epoch [30/100], Loss: 8.8954
Epoch [40/100], Loss: 29.2640
Epoch [50/100], Loss: 48.4565
Epoch [60/100], Loss: 43.5415
Epoch [70/100], Loss: 17.9307
Epoch [80/100], Loss: 47.0508
Epoch [90/100], Loss: 41.2304
Epoch [100/100], Loss: 42.1171


In [23]:
preds = model(inputs)
preds


tensor([[ 58.0348,  71.4776],
        [ 79.7019,  97.3358],
        [123.8129, 139.7469],
        [ 58.0348,  71.4776],
        [ 79.7019,  97.3358],
        [123.8129, 139.7469],
        [ 25.0967,  42.2390],
        [ 95.1756, 110.2697],
        [ 25.0967,  42.2390],
        [ 95.1756, 110.2697],
        [ 58.0348,  71.4776],
        [ 79.7019,  97.3358],
        [123.8129, 139.7469],
        [ 25.0967,  42.2390],
        [ 95.1756, 110.2697]], grad_fn=<AddmmBackward0>)

In [24]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])